In [80]:
from __future__ import print_function
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.layers import Layer
from tensorflow.keras import activations
from tensorflow.keras import utils
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

#import keras
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import math
from itertools import cycle
import numpy as np
import pandas as pd
#import seaborn as sn
from matplotlib import pyplot as plt

from utils import *
from layers import *

K.set_image_data_format('channels_last')

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  #按需分配显存
session = tf.compat.v1.Session(config=config)

tf.__version__

'2.3.0'

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
def capsnet(inputs):
    x = Conv2D(16, 5, 1, padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization(axis=3, center=True, scale=True, beta_initializer="random_uniform", gamma_initializer="random_uniform")(x)
    x = ReLU()(x)
    
    x = MaxPooling2D()(x)
    
    x = Conv2D(32, 5, 1, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization(axis=3, center=True, scale=True, beta_initializer="random_uniform", gamma_initializer="random_uniform")(x)
    x = LeakyReLU()(x)
    
    x = MaxPooling2D()(x)
    
    x = Conv2D(64, 5, 1, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization(axis=3, center=True, scale=True, beta_initializer="random_uniform", gamma_initializer="random_uniform")(x)
    x = LeakyReLU()(x)
    
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, 5, 1, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization(axis=3, center=True, scale=True, beta_initializer="random_uniform", gamma_initializer="random_uniform")(x)
    x = LeakyReLU()(x)

    
    x = PrimaryCaps_H(32, 8, 9, 1, padding='SAME')(x)

    digit_caps = DigitCaps(3, 16, 3)(x)


    digit_caps_len = Lambda(lambda z: K.sqrt(K.sum(K.square(z), 2)))(digit_caps)
    model = Model(inputs=[inputs], outputs=[digit_caps, digit_caps_len], name='CapsNet')

    return model

In [4]:
def generator(input_shape):
    inputs = Input(16*3)
    
    x = tf.keras.layers.Dense(512, activation='relu')(inputs)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(np.prod(input_shape), activation='sigmoid')(x)
    x = tf.keras.layers.Reshape(target_shape=input_shape, name='out_generator')(x)
    
    return Model(inputs=[inputs], outputs=[x], name='Generator')

In [5]:
class Mask(Layer):
    def call(self, inputs, double_mask=None, **kwargs):
        if type(inputs) is list:
            if double_mask:
                inputs, mask1, mask2 = inputs
            else:
                inputs, mask = inputs
        else:  
            x = tf.sqrt(tf.reduce_sum(tf.square(inputs), -1))
            if double_mask:
                mask1 = tf.keras.backend.one_hot(tf.argsort(x,direction='DESCENDING',axis=-1)[...,0],num_classes=x.get_shape().as_list()[1])
                mask2 = tf.keras.backend.one_hot(tf.argsort(x,direction='DESCENDING',axis=-1)[...,1],num_classes=x.get_shape().as_list()[1])
            else:
                mask = tf.keras.backend.one_hot(indices=tf.argmax(x, 1), num_classes=x.get_shape().as_list()[1])

        if double_mask:
            masked1 = tf.keras.backend.batch_flatten(inputs * tf.expand_dims(mask1, -1))
            masked2 = tf.keras.backend.batch_flatten(inputs * tf.expand_dims(mask2, -1))
            return masked1, masked2
        else:
            masked = tf.keras.backend.batch_flatten(inputs * tf.expand_dims(mask, -1))
            return masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:  # generation step
            return tuple([None, input_shape[1] * input_shape[2]])

    def get_config(self):
        config = super(Mask, self).get_config()
        return config

In [6]:
def create_model(inputs, y_true, mode='train'):
    capsnet_model = capsnet(inputs)
    digit_caps, digit_caps_len = capsnet_model(inputs)
    
    # 重构
    masked_by_y = Mask()([digit_caps, y_true])  
    masked = Mask()(digit_caps)
    
    generator_model = generator([128, 128, 3])
    x_gen_train = generator_model(masked_by_y)
    x_gen_eval = generator_model(masked)
    
    if mode == 'train':   
        return Model([inputs, y_true], [digit_caps_len, x_gen_train], name='CapsNet_Generator')
    elif mode == 'test':
        return Model(inputs, [digit_caps_len, x_gen_eval], name='CapsNet_Generator')
    else:
        raise RuntimeError('mode not recognized')

In [7]:
inputs = Input(shape=(128, 128, 3))
y_true = Input(shape=(3,))

model = create_model(inputs, y_true)

In [8]:
adam = optimizers.Adam(lr=0.0001) 

model.compile(loss=[margin_loss, 'mse'], optimizer=adam, loss_weights=[1., 0.392], metrics={'CapsNet': 'accuracy'})
model.summary()

Model: "CapsNet_Generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
CapsNet (Functional)            [(None, 3, 16), (Non 6071440     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
mask (Mask)                     (None, 48)           0           CapsNet[0][0]                    
                                                                 input_2[0][0]    

In [9]:
#learning decay rate schedule
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5 
    epochs_drop = 20  
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

In [13]:
batch_size = 16  
num_classes = 3
epochs = 100

images= np.load("data/image.npy")
labels= np.load("data/label.npy")

In [11]:
def generator_(image, label):
    return (image, label), (label, image)

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
cvscores = []
cvpre = []
cvrecall  =[]
cvf1 = []
cvauc = []

for k, (train, test) in enumerate(kfold.split(images, labels)):
    
    path = "model/"
    
    x_train = images[train]
    x_test = images[test]
    y_train = labels[train]
    y_test = labels[test]
    
#     np.save(path + 'data/x_train.npy', x_train)
#     np.save(path + 'data/y_train.npy', y_train)
#     np.save(path + 'data/x_test.npy', x_test)
#     np.save(path + 'data/y_test.npy', y_test)
    
    #class weights to handle class imbalance
    class_weights = {0: 1-np.count_nonzero(y_train==0)/len(y_train), 
                     1: 1-np.count_nonzero(y_train==1)/len(y_train), 
                     2: 1-np.count_nonzero(y_train==2)/len(y_train)}

    # 将整型标签转为onehot
    y_train = utils.to_categorical(y_train, num_classes)
    y_test = utils.to_categorical(y_test, num_classes)
    
    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_train = dataset_train.map(generator_, num_parallel_calls=16)
    dataset_train = dataset_train.batch(batch_size)
    dataset_train = dataset_train.prefetch(-1)

    dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    dataset_test = dataset_test.map(generator_, num_parallel_calls=16)
    dataset_test = dataset_test.batch(batch_size)
    dataset_test = dataset_test.prefetch(-1)
    
    
    # The best model is selected based on the loss value on the validation set
    filepath = path+"cg-weights-best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_CapsNet_accuracy',
                                 save_best_only=True, save_weights_only=True, verbose=1, mode='max')

    
    # learning schedule callback
    lrate = LearningRateScheduler(step_decay)

    callbacks_list = [checkpoint, lrate]
    

    #===================================Model======================================
    inputs = Input(shape=(128, 128, 3))
    y_true = Input(shape=(3,))
    model = create_model(inputs, y_true)
    adam = optimizers.Adam(lr=0.0001) 
    model.compile(loss=[margin_loss, 'mse'], optimizer=adam, loss_weights=[1., 0.392], metrics={'CapsNet': 'accuracy'})
    
    print("========================= 第"+ str(k+1) +"折开始 ============================")
    history = model.fit(dataset_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(dataset_test), 
                    #class_weight=class_weights, 
                    shuffle=True, 
                    callbacks=callbacks_list)
    
#     np.save(path+'acc.npy', history.history['accuracy'])
#     np.save(path+'val_acc.npy', history.history['val_accuracy'])
#     np.save(path+'loss.npy', history.history['loss'])
#     np.save(path+'val_loss.npy', history.history['val_loss'])
    
    model.load_weights(filepath)
    # evaluate the model
    #scores = model.evaluate(x_test, y_test, verbose=0)
    scores = model.evaluate(dataset_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    #predict = model.predict([x_test])
    predict = model.predict(dataset_test)[0]
    
    # ===================auc=========================
    n_classes = y_test.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], predict[:, i], )
        roc_auc[i] = auc(fpr[i], tpr[i])
    roc_auc["macro"] = roc_auc_score(y_test, predict, multi_class="ovo", average="macro")
    roc_auc["weighted"] = roc_auc_score(y_test, predict, multi_class="ovo", average="weighted")
    # ===================================================

    y_pre = np.argmax(predict, axis=1)
    y_test = np.argmax(y_test, axis=1)

    report = classification_report(y_test, y_pre, output_dict=True)
    df1 = pd.DataFrame(report).transpose()
    
    df1['auc'] = [roc_auc[0], roc_auc[1], roc_auc[2], " ", roc_auc["macro"], roc_auc["weighted"]]
    
    # Write it into csv format
    #df1.to_csv(path+'report.csv', index=True, header=True)
    
    cvpre.append(df1.loc['macro avg','precision'] * 100)
    cvrecall.append(df1.loc['macro avg','recall'] * 100)
    cvf1.append(df1.loc['macro avg','f1-score'] * 100)
    cvauc.append(df1.loc['macro avg','auc'] * 100)
    
#     data = confusion_matrix(y_test, y_pre)
#     names = ['normal', 'pneumonia', 'COVID-19']
#     df_cm = pd.DataFrame(data, columns=names, index=names)
#     df_cm.to_csv(path+'cm.csv', index=True, header=True)
    print("========================= 第"+ str(k+1) +"折结束 ============================")
    
print("accuracy：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("precision：%.2f%% (+/- %.2f%%)" % (np.mean(cvpre), np.std(cvpre)))
print("recall：%.2f%% (+/- %.2f%%)" % (np.mean(cvrecall), np.std(cvrecall)))
print("f1-score：%.2f%% (+/- %.2f%%)" % (np.mean(cvf1), np.std(cvf1)))
print("auc：%.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))

========================= 第1折开始 ============================
Epoch 1/100
50/50 [==============================] - ETA: 0s - loss: 0.8320 - CapsNet_loss: 0.8100 - Generator_loss: 0.0562 - CapsNet_accuracy: 0.2755
Epoch 00001: val_CapsNet_accuracy improved from -inf to 0.35176, saving model to model\cg-weights-best.h5
50/50 [==============================] - 7s 132ms/step - loss: 0.8320 - CapsNet_loss: 0.8100 - Generator_loss: 0.0562 - CapsNet_accuracy: 0.2755 - val_loss: 0.8314 - val_CapsNet_loss: 0.8098 - val_Generator_loss: 0.0551 - val_CapsNet_accuracy: 0.3518
Epoch 2/100
50/50 [==============================] - ETA: 0s - loss: 0.8312 - CapsNet_loss: 0.8092 - Generator_loss: 0.0560 - CapsNet_accuracy: 0.2327
Epoch 00002: val_CapsNet_accuracy did not improve from 0.35176
50/50 [==============================] - 5s 109ms/step - loss: 0.8312 - CapsNet_loss: 0.8092 - Generator_loss: 0.0560 - CapsNet_accuracy: 0.2327 - val_loss: 0.8271 - val_CapsNet_loss: 0.8056 - val_Generator_loss: 0.05

Epoch 18/100
50/50 [==============================] - ETA: 0s - loss: 0.2072 - CapsNet_loss: 0.1977 - Generator_loss: 0.0243 - CapsNet_accuracy: 0.6818
Epoch 00018: val_CapsNet_accuracy improved from 0.65829 to 0.81910, saving model to model\cg-weights-best.h5
50/50 [==============================] - 6s 115ms/step - loss: 0.2072 - CapsNet_loss: 0.1977 - Generator_loss: 0.0243 - CapsNet_accuracy: 0.6818 - val_loss: 0.1964 - val_CapsNet_loss: 0.1872 - val_Generator_loss: 0.0236 - val_CapsNet_accuracy: 0.8191
Epoch 19/100
50/50 [==============================] - ETA: 0s - loss: 0.1864 - CapsNet_loss: 0.1769 - Generator_loss: 0.0243 - CapsNet_accuracy: 0.7195
Epoch 00019: val_CapsNet_accuracy improved from 0.81910 to 0.82412, saving model to model\cg-weights-best.h5
50/50 [==============================] - 6s 116ms/step - loss: 0.1864 - CapsNet_loss: 0.1769 - Generator_loss: 0.0243 - CapsNet_accuracy: 0.7195 - val_loss: 0.2617 - val_CapsNet_loss: 0.2523 - val_Generator_loss: 0.0238 - val_C

Epoch 35/100
50/50 [==============================] - ETA: 0s - loss: 0.0685 - CapsNet_loss: 0.0590 - Generator_loss: 0.0242 - CapsNet_accuracy: 0.9421
Epoch 00035: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 109ms/step - loss: 0.0685 - CapsNet_loss: 0.0590 - Generator_loss: 0.0242 - CapsNet_accuracy: 0.9421 - val_loss: 0.0878 - val_CapsNet_loss: 0.0786 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.9196
Epoch 36/100
50/50 [==============================] - ETA: 0s - loss: 0.0658 - CapsNet_loss: 0.0563 - Generator_loss: 0.0242 - CapsNet_accuracy: 0.9434
Epoch 00036: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 109ms/step - loss: 0.0658 - CapsNet_loss: 0.0563 - Generator_loss: 0.0242 - CapsNet_accuracy: 0.9434 - val_loss: 0.0870 - val_CapsNet_loss: 0.0778 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.9246
Epoch 37/100
50/50 [==============================] - ETA: 0s - loss

Epoch 53/100
50/50 [==============================] - ETA: 0s - loss: 0.0369 - CapsNet_loss: 0.0274 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9786
Epoch 00053: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 109ms/step - loss: 0.0369 - CapsNet_loss: 0.0274 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9786 - val_loss: 0.0837 - val_CapsNet_loss: 0.0745 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.8995
Epoch 54/100
50/50 [==============================] - ETA: 0s - loss: 0.0361 - CapsNet_loss: 0.0267 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9786
Epoch 00054: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 109ms/step - loss: 0.0361 - CapsNet_loss: 0.0267 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9786 - val_loss: 0.0836 - val_CapsNet_loss: 0.0744 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.8995
Epoch 55/100
50/50 [==============================] - ETA: 0s - loss

Epoch 71/100
50/50 [==============================] - ETA: 0s - loss: 0.0274 - CapsNet_loss: 0.0180 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9899
Epoch 00071: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 110ms/step - loss: 0.0274 - CapsNet_loss: 0.0180 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9899 - val_loss: 0.0804 - val_CapsNet_loss: 0.0712 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.9095
Epoch 72/100
50/50 [==============================] - ETA: 0s - loss: 0.0272 - CapsNet_loss: 0.0177 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9899
Epoch 00072: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 110ms/step - loss: 0.0272 - CapsNet_loss: 0.0177 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9899 - val_loss: 0.0803 - val_CapsNet_loss: 0.0711 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.9045
Epoch 73/100
50/50 [==============================] - ETA: 0s - loss

Epoch 89/100
50/50 [==============================] - ETA: 0s - loss: 0.0237 - CapsNet_loss: 0.0143 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9937
Epoch 00089: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 109ms/step - loss: 0.0237 - CapsNet_loss: 0.0143 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9937 - val_loss: 0.0790 - val_CapsNet_loss: 0.0698 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.8995
Epoch 90/100
50/50 [==============================] - ETA: 0s - loss: 0.0236 - CapsNet_loss: 0.0142 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9937
Epoch 00090: val_CapsNet_accuracy did not improve from 0.93467
50/50 [==============================] - 5s 109ms/step - loss: 0.0236 - CapsNet_loss: 0.0142 - Generator_loss: 0.0241 - CapsNet_accuracy: 0.9937 - val_loss: 0.0790 - val_CapsNet_loss: 0.0698 - val_Generator_loss: 0.0235 - val_CapsNet_accuracy: 0.8995
Epoch 91/100
50/50 [==============================] - ETA: 0s - loss

TypeError: list indices must be integers or slices, not tuple

In [64]:
X = np.load("newData/image.npy")
Y = np.load("newData/label.npy")

y = utils.to_categorical(Y, num_classes)
    
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.map(generator_, num_parallel_calls=16)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(-1)

In [81]:
#predict = model.predict(dataset)[0]
predict = np.array([])
for i, data in enumerate(dataset):
    if i == 0:
        predict = model.predict(data)[0]
    else:
        predict = np.concatenate((predict, model.predict(data)[0]))

y = label_binarize(Y, classes=[0, 1, 2])
n_classes = y.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], predict[:, i], )
    roc_auc[i] = auc(fpr[i], tpr[i])
roc_auc["macro"] = roc_auc_score(y, predict, multi_class="ovo", average="macro")
roc_auc["weighted"] = roc_auc_score(y, predict, multi_class="ovo", average="weighted")


y_pre = np.argmax(predict, axis=1)
report = classification_report(Y, y_pre, output_dict=True)
df1 = pd.DataFrame(report).transpose()
df1['auc'] = [roc_auc[0], roc_auc[1], roc_auc[2], '', roc_auc["macro"], roc_auc["weighted"]]
df1

,precision,recall,f1-score,support,auc
0,0.852162,0.911260,0.880721,1341.000000,0.951683
1,0.874308,0.822305,0.847510,1345.000000,0.938098
2,0.947767,0.937500,0.942606,1200.000000,0.987714
accuracy,0.888574,0.888574,0.888574,0.888574,
macro avg,0.891413,0.890355,0.890279,3886.000000,0.959165
weighted avg,0.889350,0.888574,0.888336,3886.000000,0.958107
